## 텍스트 벡터화 직접 구현하기

이 노트북은 텍스트 데이터를 벡터로 변환하는 과정을 직접 구현하는 `MyVectorize` 클래스를 다룹니다. 텍스트 표준화, 토큰화, 어휘 사전 구축, 인코딩 및 디코딩 기능을 포함합니다.

### 1. 라이브러리 임포트
필요한 라이브러리를 임포트합니다. 여기서는 구두점 처리를 위해 `string` 모듈을 사용합니다.

In [ ]:
import string

### 2. MyVectorize 클래스 정의
텍스트 처리를 위한 모든 메서드를 포함하는 클래스입니다.
- `standardize`: 텍스트를 소문자로 변환하고 구두점을 제거합니다.
- `tokenize`: 텍스트를 공백 기준으로 단어 토큰으로 분리합니다.
- `make_vocabulary`: 데이터셋으로부터 단어 사전을 구축합니다.
- `encode`: 텍스트를 숫자 시퀀스로 변환합니다.
- `decode`: 숫자 시퀀스를 다시 텍스트로 변환합니다.

In [ ]:
class MyVectorize:
    def standardize(self, text): #표준화
        text = text.lower() #1.전부 소문자로 만든다 
        return "".join(c for c in text  if c not in string.punctuation)
        #구두점 제거한 문장을 만들어서 반환함 
    
    def tokenize(self, text): #토큰화 
        return text.split()  #잘라서 보낸다. 

    #어휘사전 만드는 함수 
    def make_vocabulary(self, dataset):
        #1.전체 데이터셋을 순회하녀 단어 사전을 만든다. 
        #2.기본저긍로 빈문장"" 과 UNK- 자주 사용하는 단어만, 자주 사용하는 단어가 아니면 UNK로 표현한다.(unknown)
        #3.새로운 단어가 발견되면 어휘사전에 추가하고 해당 단어에 고유한 숫자 인데스를 부여한다 
        self.vocabulary = {"":0, "[UNK]":1}  #0하고 1은 특수목적으로 사용한다 
        for text in dataset: #한문장씩 처리한다
            text = self.standardize(text) #표준화 
            tokens = self.tokenize(text)  #토큰화 
            for token in tokens:
                if token not in self.vocabulary: #아직 어휘사전에 없는 단어이면 추가한다 
                    self.vocabulary[token] = len(self.vocabulary)

        #역순서  단어:숫자 => 숫자:단어로 바꾼다
        self.inverse_vocabulary = dict( (v,k) for k,v in self.vocabulary.items())  

    def encode(self, text): #문장을 받아서 문장 벡터화를 한다 
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        return " ".join(self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

### 3. 데이터 준비 및 클래스 인스턴스화
실습에 사용할 간단한 데이터셋을 정의하고 `MyVectorize` 클래스의 인스턴스를 생성합니다.

In [ ]:
mv = MyVectorize()
dataset=[
    "I write, erase, reqrite",
    "Erase again, and then",
    "A poppy blooms",
    "Dog is pretty"
]

### 4. 표준화 및 토큰화 테스트
`standardize`와 `tokenize` 메서드가 잘 동작하는지 확인합니다.

In [ ]:
test_sentence = dataset[1]
print(f"Original: {test_sentence}")

standardized_test = mv.standardize(test_sentence)
print(f"Standardized: {standardized_test}")

tokenized_test = mv.tokenize(standardized_test)
print(f"Tokenized: {tokenized_test}")

### 5. 어휘 사전 생성
`make_vocabulary` 메서드를 호출하여 전체 데이터셋에 대한 어휘 사전을 생성합니다. 생성된 `vocabulary` (단어 -> 인덱스)와 `inverse_vocabulary` (인덱스 -> 단어)를 출력하여 확인합니다.

In [ ]:
mv.make_vocabulary(dataset)

print("Vocabulary (word to index):")
print(mv.vocabulary)

print("\nInverse Vocabulary (index to word):")
print(mv.inverse_vocabulary)

### 6. 인코딩 및 디코딩 테스트
임의의 문장을 `encode` 하여 숫자 시퀀스로 변환하고, 그 숫자 시퀀스를 다시 `decode` 하여 원본과 유사한 문장으로 복원되는지 확인합니다.

In [ ]:
encoded_text = mv.encode("I write erase")
print(f"Encoded 'I write erase': {encoded_text}")

# [UNK]를 포함한 디코딩 예시 (어휘 사전에 없는 23번 인덱스)
decoded_text = mv.decode([2, 3, 4, 23])
print(f"Decoded [2, 3, 4, 23]: {decoded_text}")